In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6759,2024-04-08,Alemanha Bbl,15:00,Hamburg,Heidelberg,1.24,4.15,176.5,1.86,1.94,-10.5,2.03,10.50,MkwLqQ5T,0.806452,0.240964,0.537634,0.515464,0.047415,0.4,0.4,NaN,NaN,207.728,58.841482,0.283262,291.206,213.472754,0.733064,140.382,552.618,150.15,364.37,0.0,0.0,0.0,0.0,0.763518,0.029773,0.955977,-0.84,-0.168,-1.428571,0.00000,0.0,0.00000,-1.62,-0.324,-9.722222,0.000000,0.0,0.000000
6760,2024-04-08,Bolívia Libobasquet,20:00,Kinwa,Basquetbol La Paz,1.61,2.22,165.5,1.87,1.79,-3.5,1.93,1.92,UXVa3Kqr,0.621118,0.450450,0.534759,0.558659,0.071568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.225240,0.030912,0.003673,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
6761,2024-04-08,Coreia Do Sul Kbl,07:00,KCC Egis,Seoul Knights,1.33,3.29,168.5,1.90,1.88,-7.5,1.89,7.00,YcAbQmOF,0.751880,0.303951,0.526316,0.531915,0.055831,1.0,0.6,NaN,NaN,191.396,43.776680,0.228723,155.530,23.389917,0.150388,196.528,170.168,157.08,193.55,0.0,0.0,0.0,0.0,0.599969,0.007483,0.812894,2.07,0.414,0.797101,0.61492,0.8,0.18508,-5.00,-1.000,-2.290000,0.526308,0.5,-0.026308
6762,2024-04-08,Dinamarca Liga De Basquete,13:30,Randers,Næstved,1.82,1.93,185.5,1.80,1.86,-1.5,1.93,2.09,vmP11U1Q,0.549451,0.518135,0.555556,0.537634,0.067585,0.0,0.4,NaN,NaN,272.702,261.718152,0.959722,185.688,100.376181,0.540564,0.000,136.652,124.32,174.42,0.0,0.0,0.0,0.0,0.041484,0.023184,0.056287,0.00,0.000,inf,0.00000,0.0,0.00000,0.04,0.008,116.250000,0.000000,0.0,0.000000
6763,2024-04-08,Suécia Basketligan,14:04,Nassjo,BC Lulea,2.03,1.74,175.5,1.87,1.79,1.5,1.88,1.60,bqrgcRw1,0.492611,0.574713,0.534759,0.558659,0.067323,0.8,0.6,NaN,NaN,171.428,57.664203,0.336376,126.380,36.009931,0.284934,169.352,142.564,210.00,116.39,0.0,0.0,0.0,0.0,0.108786,0.030912,0.113787,-0.36,-0.072,-14.305556,0.00000,0.0,0.00000,-1.30,-0.260,-2.846154,0.000000,0.0,0.000000
6764,2024-04-08,Argentina Liga A,20:30,Gimnasia,Independiente de Oliva,1.28,3.44,162.5,1.85,1.85,-7.5,1.81,6.50,2m11K58d,0.781250,0.290698,0.540541,0.540541,0.071948,0.4,0.8,NaN,NaN,126.482,18.641470,0.147384,130.210,34.582754,0.265592,131.754,206.626,141.93,176.88,0.0,0.0,0.0,0.0,0.647182,0.000000,0.798154,2.41,0.482,0.580913,0.00000,0.0,0.00000,-1.06,-0.212,-11.509434,0.000000,0.0,0.000000
6765,2024-04-08,Finlândia I Divisão A,12:30,ToPo,Jyvaskyla,1.75,1.98,172.5,1.85,1.85,-2.5,1.95,2.37,fTOqDwla,0.571429,0.505051,0.540541,0.540541,0.076479,0.8,0.0,NaN,NaN,154.532,77.841501,0.503724,164.202,122.951161,0.748780,129.228,0.000,132.25,381.60,0.0,0.0,0.0,0.0,0.087204,0.000000,0.137493,-0.49,-0.098,-7.653061,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
6766,2024-04-08,Porto Rico Bsn,21:00,Cangrejeros,Osos de Manati,1.63,2.19,181.5,1.83,1.87,-2.5,1.80,2.37,buCd8bdn,0.613497,0.456621,0.546448,0.534759,0.070118,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.207319,0.015289,0.193310,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
6767,2024-04-08,Uruguai Liga Uruguaia,21:15,Nacional,Aguada,2.12,1.67,171.5,1.85,1.85,1.5,1.96,1.51,0QRaf6Vh,0.471698,0.598802,0.540541,0.540541,0.070501,0.2,0.6,NaN,NaN,126.772,35.508254,0.280095,159.366,35.053299,0.219955,123.350,170.996,167.48,215.83,0.0,0.0,0.0,0.0,0.167915,0.000000,0.183399,-1.22,-0.244,-4.590164,0.00000,0.0,0.00000,1.28,0.256,2.617188,0.616737,0.7,0.083263


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:00,Alemanha Bbl,Hamburg,Heidelberg,176.5,1.86,1.0000,Over
1,20:00,Bolívia Libobasquet,Kinwa,Basquetbol La Paz,165.5,1.87,0.9988,Over
2,07:00,Coreia Do Sul Kbl,KCC Egis,Seoul Knights,168.5,1.90,1.0000,Over
3,13:30,Dinamarca Liga De Basquete,Randers,Næstved,185.5,1.80,0.9942,Over
4,20:30,Argentina Liga A,Gimnasia,Independiente de Oliva,162.5,1.85,1.0000,Over
5,12:30,Finlândia I Divisão A,ToPo,Jyvaskyla,172.5,1.85,1.0000,Over
6,21:00,Porto Rico Bsn,Cangrejeros,Osos de Manati,181.5,1.83,1.0000,Over
